In [1]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2wpkh_script, p2pkh_script
from src.ecdsa import PrivateKey
from src.crypto import hash160

In [2]:
test = TestShell().setup(num_nodes=2, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_p2wpkh_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2wpkh_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2wpkh_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_address_1 = test.nodes[1].getnewaddress("")
bob_private_key_wif_format = bob.dumpprivkey(bob_address_1)
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

2024-03-10T13:24:22.634000Z TestFramework (INFO): PRNG seed is: 7169386173763511158
2024-03-10T13:24:22.635000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_clv6dzqu


In [3]:
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_previous_tx_id_to_spent = alice_p2wpkh_funding_tx_parsed["txid"]

for tx_out in alice_p2wpkh_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

In [4]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2wpkh_script(hash160(bob_public_key.sec_format()))
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2WPK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2WPK Lockingtransaktion: 0100000001c2e22ac65204ff35289a7c5b61aec939aa234adabfb656d8c71dcce9ceeea2680100000000ffffffff01102700000000000016001443ce34a7a2f626ef4a2f42dca420553f93600035ffffffff


In [5]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2WPKH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2wpkh_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

print(alice_p2wpkh_locking_tx)

Alice signierte P2WPKH Lockingtransaktion: 0100000001c2e22ac65204ff35289a7c5b61aec939aa234adabfb656d8c71dcce9ceeea268010000006a47304402207e1541ae1f9929c44e0d8f5859ad9f4e8897b203f8ec8ea8235de85748855e2f02206fa84bab0e0d75fb377cb7a5c9640a6db300fca77f6a83281a692b0fad992e7c012102bd150eb5f3b2cb7670b7655d755ee2a0b7671822af059730f59bbe62cfe1f970ffffffff01102700000000000016001443ce34a7a2f626ef4a2f42dca420553f93600035ffffffff
b911e0ad1ef4886feb2cc4df6384762b77ef242a69f5a57cb3178036975c9b2e


In [6]:
alice_p2wpk_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2wpkh_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2wpk_locking_tx_parsed["txid"]

for tx_out in alice_p2wpk_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

In [7]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pkh_script(hash160(alice_public_key.sec_format()))
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=True)

print(f"Bob unsignierte P2WPKH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob unsignierte P2WPKH Spendingtransaktion: 010000000001012e9b5c97368017b37ca5f5692a24ef772b768463dfc42ceb6f88f41eade011b90000000000ffffffff0128230000000000001976a91464b8bfacca8fafd12c3ff8df5e3909be0230b03188ac00ffffffff


In [8]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
print(bob_script_sig.serialize_script().hex())
bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format], bob_script_sig, input_amount=10000)

print(f"Bob signierte P2PKH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

bob_p2pkh_spending_tx = bob.sendrawtransaction(bob_spending_transaction.serialize_transaction().hex())

print(bob_p2pkh_spending_tx)

16001443ce34a7a2f626ef4a2f42dca420553f93600035
Bob signierte P2PKH Spendingtransaktion: 010000000001012e9b5c97368017b37ca5f5692a24ef772b768463dfc42ceb6f88f41eade011b90000000000ffffffff0128230000000000001976a91464b8bfacca8fafd12c3ff8df5e3909be0230b03188ac02473044022004b270486fc58de37614ead2e818eab91d4e4beb043e912927fd0454acb1149802205b06c56e2c60f51c07c1ad7b8d05f7131ca4368b737ecbfa881cc13c46ac19d0012103056ed26138a7b846974fb212babd5fe8581ea71d1defb112a7f94b3275fe1d2cffffffff
976ab1ffbb20775c89261391271046f30960fd821f0b6fe7bac224b984ae8a0d


In [9]:
TestShell().shutdown()

2024-03-10T13:24:23.610000Z TestFramework (INFO): Stopping nodes
2024-03-10T13:24:23.662000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_clv6dzqu on exit
2024-03-10T13:24:23.663000Z TestFramework (INFO): Tests successful
